Primeramente vamos a inicializar los paquetes necesarios

In [ ]:
from autoscript_tem_microscope_client import TemMicroscopeClient
from autoscript_tem_microscope_client.enumerations import *
from autoscript_tem_microscope_client.structures import *

In [ ]:
import time # If it's not necessary delete

In [ ]:
# Spiral Grid Generatod, copied from the Hackathon project
def build_spiral_coordinates(total_cells = 12, cell_size=1024, start_point=None):
    coord_initial = []
    directions = [(0, cell_size), (-cell_size, 0), (0, -cell_size), (cell_size, 0)]
    direction_index = 0

    x, y = start_point

    step_count = 0
    step_limit = 1
    direction_changes = 0

    while len(coord_initial) < total_cells:
        x_start, y_start = x, y

        coord_initial.append((x_start, y_start))

        dx, dy = directions[direction_index]
        x += dx
        y += dy
        step_count += 1

        if step_count == step_limit:
            step_count = 0
            direction_index = (direction_index + 1) % 4
            direction_changes += 1
            
            if direction_changes % 2 == 0:
                step_limit += 1

    return coord_initial

In [ ]:
def mainPrueba():   
    microscope = TemMicroscopeClient()
    microscope.connect()

    # Change optical mode to STEM
    microscope.optics.optical_mode = OpticalMode.STEM
    
    # For taking images only the spiral, not the movement ||| Don't delete
    cell_size = 0.001
    num_steps = 5
    step_size = cell_size/num_steps

    position = microscope.specimen.stage.position
    start_x, start_y = position.x, position.y
    grid_coords = build_spiral_coordinates(total_cells=12, cell_size=cell_size, start_point=(start_x, start_y))
    
    for (x, y) in grid_coords:
        # time.sleep(1) # Time to let the microscope move correctly (maybe we should put more)
        movement_capture(x, y, num_steps, step_size) # <-------------------------------------------------Change this
    
    microscope.disconnect()

In [ ]:
# Function where the microscope moves inside of the grid and takes pictres each step
def movement_capture(grid_x, grid_y, num_steps = 5, step_size=0.0001):
    
    microscope.specimen.stage.absolute_move(StagePosition(x=grid_x, y=grid_y)) # Move the microscope to the coordinates of the grid
    
    for j in range(num_steps):
        if j % 2 == 0: # First we select the direction where we are panning
            direction = 1
        else:
            direction = -1

        print(f"Step {j}")
        
        for i in range(num_steps):
            image = microscope.camera.acquire_image() # HAADF
            image.save(f"image_{j}_{i}.png")
            microscope.specimen.stage.relative_move(StagePosition(x=direction * step_size))
            # time.sleep(1) # Time to let the microscope move correctly (maybe we should put more)
            
        microscope.specimen.stage.relative_move(StagePosition(y=-step_size))  

In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
def main():
    microscope = TemMicroscopeClient()
    microscope.connect()
    microscope.optics.optical_mode = OpticalMode.STEM

    num_images = 1000
    step_size = 0.0001

    position = microscope.specimen.stage.position
    start_x, start_y = position.x, position.y
    grid_coords = build_spiral_coordinates(total_cells=num_images/2, cell_size=step_size, start_point=(start_x, start_y))

    for (x, y) in grid_coords:
        movement(x, y, step_size, microscope)
    
    microscope.disconnect()

In [ ]:
def movement(grid_x, grid_y, step_size, microscope):
    microscope.specimen.stage.absolute_move(StagePosition(x=grid_x, y=grid_y)) # Move the microscope to the coordinates of the grid
    for i in range (0,2):
            image = microscope.acquisition.acquire_stem_image(DetectorType.HAADF, ImageSize.PRESET_512, 1e-6)
            image.save(f"image_{grid_x}_{i}.png")
            microscope.specimen.stage.relative_move(StagePosition(x=step_size/2))